In [1]:
!unzip fer2013.zip

Archive:  fer2013.zip
  inflating: fer2013.csv             


In [2]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate, Conv2DTranspose, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.utils import plot_model
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import imgaug as ia
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Load Dataset

In [3]:
data = pd.read_csv('./fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
data.emotion.value_counts()

3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: emotion, dtype: int64

In [5]:
dic = {0:'Angry', 1:'disgust' , 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}

The emotion disgust has too few samples, therefore we won't use it

In [6]:
data = data[data.emotion != 1]
data['emotion'] = data.emotion.replace(6, 1)

data.emotion.value_counts()

3    8989
1    6198
4    6077
2    5121
0    4953
5    4002
Name: emotion, dtype: int64

We will redefine the dictionary

In [7]:
dic = {0:'Angry', 1:'Neutral', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise'}

In [8]:
num_classes = 6
img_width = 48
img_height = 48

In [9]:
X = data['pixels']
y = data['emotion']

X_train = []
for i in X:
    X_train.append([int(j) for j in i.split()])

X_train = np.array(X_train)/255.0

X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_train = X_train.astype('float32')

y_train = y.to_numpy().reshape(-1, 1)

print(X_train.shape)

(35340, 48, 48, 1)


## Data Augmentation

In [10]:
epochs = X_train.shape[0]
print("number of epochs:", epochs)

X_train_aug = X_train
y_train_aug = y_train

for k in range(epochs):
  img = X_train[k]
  emotion = y_train[k]
  contrasted_images = []
  emotions_list = []

  # contrast = iaa.GammaContrast(gamma=2.0)
  contrast = iaa.Affine(rotate=(-50, 30))
  contrast_image = contrast.augment_image(img)
  contrasted_images.append(contrast_image)

  contrasted_images = np.array(contrasted_images)
  emotions_list = np.array(emotions_list)
  X_train_aug = np.concatenate((X_train_aug, contrasted_images), axis=0)
  emotions_list = [emotion]
  y_train_aug = np.concatenate((y_train_aug, emotions_list), axis=0)

  if k % 100 == 0:
    print ("iteration:" , k,", train shape:", X_train_aug.shape)
    # print ("iteration: %d, train shape: %s" % (k, X_train_aug.shape))

number of epochs: 35340
iteration: 0 , train shape: (35341, 48, 48, 1)
iteration: 100 , train shape: (35441, 48, 48, 1)
iteration: 200 , train shape: (35541, 48, 48, 1)
iteration: 300 , train shape: (35641, 48, 48, 1)
iteration: 400 , train shape: (35741, 48, 48, 1)
iteration: 500 , train shape: (35841, 48, 48, 1)
iteration: 600 , train shape: (35941, 48, 48, 1)
iteration: 700 , train shape: (36041, 48, 48, 1)
iteration: 800 , train shape: (36141, 48, 48, 1)
iteration: 900 , train shape: (36241, 48, 48, 1)
iteration: 1000 , train shape: (36341, 48, 48, 1)
iteration: 1100 , train shape: (36441, 48, 48, 1)
iteration: 1200 , train shape: (36541, 48, 48, 1)
iteration: 1300 , train shape: (36641, 48, 48, 1)
iteration: 1400 , train shape: (36741, 48, 48, 1)
iteration: 1500 , train shape: (36841, 48, 48, 1)
iteration: 1600 , train shape: (36941, 48, 48, 1)
iteration: 1700 , train shape: (37041, 48, 48, 1)
iteration: 1800 , train shape: (37141, 48, 48, 1)
iteration: 1900 , train shape: (37241,

In [11]:
print("X_train_aug shape:", X_train_aug.shape)
print("y_train_aug shape:", y_train_aug.shape)

X_train_aug shape: (70680, 48, 48, 1)
y_train_aug shape: (70680, 1)


## BiCoGAN Model

In [12]:
class BiCoGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 48
        self.img_cols = 48
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 6
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        print(self.discriminator.summary())
        plot_model(self.discriminator, show_shapes=True)
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        label = Input(shape=(1,))

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim, ))
        img_ = self.generator([z, label])

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_, label])
        valid = self.discriminator([z_, img, label])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bicogan_generator = Model([z, img, label], [fake, valid])
        self.bicogan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
            optimizer=optimizer)

    def build_encoder(self):
        model = Sequential()

        # model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(Dropout(0.25))
        # model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        # model.add(BatchNormalization(momentum=0.8))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(Dropout(0.25))
        # model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        # model.add(BatchNormalization(momentum=0.8))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(Dropout(0.25))
        # model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        # model.add(BatchNormalization(momentum=0.8))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(Dropout(0.25))
        # model.add(Flatten())
        # model.add(Dense(self.latent_dim))


        model.add(Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=self.img_shape))
        model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(Conv2D(256, (5,5), strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(Conv2D(512, (5,5), strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(Flatten())
        model.add(Dense(self.latent_dim))

        # model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', input_shape=self.img_shape))
        # model.add(Conv2D(128, (4,4), strides=(2,2), padding='same'))
        # model.add(BatchNormalization(momentum=0.9))
        # model.add(Conv2D(256, (4,4), strides=(2,2), padding='same'))
        # model.add(BatchNormalization(momentum=0.9))
        # model.add(Conv2D(512, (4,4), strides=(2,2), padding='same'))
        # model.add(BatchNormalization(momentum=0.9))
        # model.add(Flatten())
        # model.add(Dense(self.latent_dim))

        # model.add(Flatten(input_shape=self.img_shape))
        # model.add(Dense(512))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(BatchNormalization(momentum=0.8))
        # model.add(Dense(512))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(BatchNormalization(momentum=0.8))
        # model.add(Dense(self.latent_dim))

        print('encoder')
        model.summary()

        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):

        model = Sequential()
        # foundation for 12x12 image
        n_nodes = 128 * 12 * 12
        model.add(Dense(n_nodes, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Reshape((12, 12, 128)))
        # upsample to 24x24
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # upsample to 48x48
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # generate
        model.add(Conv2D(1, (12, 12), activation='tanh', padding='same'))

        # model = Sequential()
        # # foundation for 3x3 feature maps
        # n_nodes = 128 * 3 * 3
        # model.add(Dense(n_nodes, input_dim=self.latent_dim))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(Reshape((3, 3, 128)))
        # # upsample to 6x6
        # model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        # model.add(LeakyReLU(alpha=0.2))
        # # upsample to 12x12
        # model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        # model.add(LeakyReLU(alpha=0.2))
        # # upsample to 24x24
        # model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        # model.add(LeakyReLU(alpha=0.2))
        # # upsample to 48x48
        # model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        # model.add(LeakyReLU(alpha=0.2))
        # # output layer 48x48x1
        # model.add(Conv2D(1, (3,3), activation='tanh', padding='same'))

        # model.add(Dense(256, input_dim=self.latent_dim))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(BatchNormalization(momentum=0.8))
        # model.add(Dense(512))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(BatchNormalization(momentum=0.8))
        # model.add(Dense(1024))
        # model.add(LeakyReLU(alpha=0.2))
        # model.add(BatchNormalization(momentum=0.8))
        # model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        # model.add(Reshape(self.img_shape))

        print('generator')
        model.summary()

        z = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([z, label_embedding])
        img = model(model_input)

        return Model([z, label], img)

    # def build_discriminator(self):

    #     z = Input(shape=(self.latent_dim, ))
    #     img = Input(shape=self.img_shape)
    #     label = Input(shape=(1,), dtype='int32')

    #     label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
    #     flat_img = Flatten()(img)

    #     d_in = concatenate([z, flat_img, label_embedding])

    #     model = Dense(1024)(d_in)
    #     model = LeakyReLU(alpha=0.2)(model)
    #     model = Dropout(0.5)(model)
    #     model = Dense(1024)(model)
    #     model = LeakyReLU(alpha=0.2)(model)
    #     model = Dropout(0.5)(model)
    #     model = Dense(1024)(model)
    #     model = LeakyReLU(alpha=0.2)(model)
    #     model = Dropout(0.5)(model)
    #     validity = Dense(1, activation="sigmoid")(model)

    #     return Model([z, img, label], validity, name='discriminator')

    def build_discriminator(self):
        xi = Input(self.img_shape)
        zi = Input(self.latent_dim)
        label = Input(shape=(1,), dtype='int32')

        # xn = Conv2D(df_dim, (5, 5), (2, 2), act=lrelu, W_init=w_init)(xi)
        # xn = Conv2D(df_dim * 2, (5, 5), (2, 2), W_init=w_init, b_init=None)(xn)
        # xn = BatchNorm2d(decay=0.9, act=lrelu, gamma_init=g_init)(xn)
        # xn = Dropout(keep=0.8)(xn)
        # xn = Conv2d(df_dim * 4, (5, 5), (2, 2), W_init=w_init, b_init=None)(xn)
        # xn = BatchNorm2d(decay=0.9, act=lrelu, gamma_init=g_init)(xn)
        # xn = Dropout(keep=0.8)(xn)
        # xn = Conv2d(df_dim * 8, (5, 5), (2, 2), W_init=w_init, b_init=None)(xn)
        # xn = BatchNorm2d(decay=0.9, act=lrelu, gamma_init=g_init)(xn)
        # xn = Dropout(keep=0.8)(xn)
        # xn = Flatten()(xn)

        xn = Conv2D(128, (5,5), padding='same')(xi)
        xn = LeakyReLU(alpha=0.2)(xn)
        # downsample to 24x24
        xn = Conv2D(128, (5,5), strides=(2,2), padding='same')(xn)
        xn = LeakyReLU(alpha=0.2)(xn)
        # downsample to 12x12
        xn = Conv2D(128, (5,5), strides=(2,2), padding='same')(xn)
        xn = LeakyReLU(alpha=0.2)(xn)
        # downsample to 6x6
        xn = Conv2D(128, (5,5), strides=(2,2), padding='same')(xn)
        xn = LeakyReLU(alpha=0.2)(xn)
        # downsample to 3x3
        xn = Conv2D(128, (5,5), strides=(2,2), padding='same')(xn)
        xn = LeakyReLU(alpha=0.2)(xn)
        # classifier
        xn = Flatten()(xn)

        zn = Flatten()(zi)
        # zn = Dense(512, activation='relu')(zn)
        # zn = Dropout(0.2)(zn)

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))

        nn = concatenate([zn, xn, label_embedding])
        nn = Dense(1, activation='sigmoid')(nn)

        return Model([zi, xi, label], nn, name='discriminator')


    def train(self, epochs, batch_size=128, sample_interval=50):
        
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images and encode
            idx = np.random.randint(0, X_train_aug.shape[0], batch_size)
            imgs, labels = X_train_aug[idx], y_train_aug[idx]
            z_ = self.encoder.predict(imgs)

            # Sample noise and generate img
            z = np.random.normal(0, 1, (batch_size, 100))
            imgs_ = self.generator.predict([z, labels])

            # Train the discriminator (img -> z is valid, z -> img is fake)
            d_loss_real = self.discriminator.train_on_batch([z_, imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([z, imgs_, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(0, 6, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.bicogan_generator.train_on_batch([z, imgs, sampled_labels], [valid, fake])

            # Plot the progress
            if epoch%20 == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)


    def sample_images(self, epoch):
          r, c = 1, 6
          noise = np.random.normal(0, 1, (r * c, 100))
          sampled_labels = np.arange(0, 6).reshape(-1, 1)

          gen_imgs = self.generator.predict([noise, sampled_labels])

          # Rescale images 0 - 1
          gen_imgs = 0.5 * gen_imgs + 0.5

          fig, axs = plt.subplots(r, c)
          cnt = 0
          for j in range(c):
              axs[j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
              axs[j].set_title("%s" % dic[sampled_labels[cnt][0]])
              axs[j].axis('off')
              cnt += 1
          fig.savefig("images/%d.png" % epoch)
          plt.close()


In [13]:
if __name__ == '__main__':
    bicogan = BiCoGAN()
    bicogan.train(epochs=18610, batch_size=128, sample_interval=200)

Model: "discriminator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 48, 48, 128)  3328        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 48, 48, 128)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 24, 24, 128)  409728      leaky_re_lu[0][0]                
______________________________________________________________________________________